In [65]:
#!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastai.vision.all import *
from fastbook import *
from graphviz import Digraph

matplotlib.rc('image', cmap='Greys')

In [66]:
class BasicOptim:
    def __init__(self,param,lr):
        self.param,self.lr = list(param),lr
    
    def step(self, *args, **kwargs):
        for p in self.param:
            p.data -= p.grad.data*self.lr
    
    def zero_grad(self, *args, **kwargs):
        for p in self.param:
            p.grad = None
        

In [68]:
linear_model = nn.Linear(28*28,1)
ls=0.1
opt = BasicOptim(linear_model.parameters(),ls)


In [97]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

def mnist_loss(pred, target): 
    pred = pred.sigmoid()
    return torch.where(target == 1, 1-pred, pred).mean()

def train_epoch(model):
    for xb, yb in train_dl:
        calc_grad(xb,yb,model)
        opt.step()
        opt.zero_grad()
    
def train_model(model, epochs):
    for i in range(epochs):
        train_epoch(model)
        print(validate_epoch(model),end=' ')

def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds,yb)
    loss.backward()

def validate_epoch(model):
    accs = [batch_accuracy(model(xb),yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(),4)

def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds > 0.5) == yb
    return correct.float().mean()


In [98]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path
path.ls()

(#4) [Path('valid'),Path('labels.csv'),Path('.DS_Store'),Path('train')]

In [99]:
#Train
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

three_tensor = [tensor(Image.open(o)) for o in threes]
seven_tensor = [tensor(Image.open(o)) for o in sevens]
stack_threes = torch.stack(three_tensor).float()/255
stack_seven = torch.stack(seven_tensor).float()/255

train_x = torch.cat([stack_threes, stack_seven]).view(-1,28*28)
train_y = tensor([1]*len(threes)+[0]*len(sevens)).unsqueeze(1)


#Valid
threes_v = (path/'valid'/'3').ls().sorted()
sevens_v = (path/'valid'/'7').ls().sorted()

threev_tensor = [tensor(Image.open(o)) for o in threes_v]
sevenv_tensor = [tensor(Image.open(o)) for o in sevens_v]
stackv_threes = torch.stack(threev_tensor).float()/255
stackv_seven = torch.stack(sevenv_tensor).float()/255

valid_x = torch.cat([stackv_threes, stackv_seven]).view(-1,28*28)
valid_y = tensor([1]*len(threes_v)+[0]*len(sevens_v)).unsqueeze(1)
train_dset = list(zip(train_x,train_y))
valid_dset = list(zip(valid_x,valid_y))

In [100]:
train_dl = DataLoader(train_dset,batch_size=200)
valid_dl = DataLoader(valid_dset,batch_size=200)

In [101]:
weights = init_params(28*28,1)
bias = init_params(1)

In [102]:
train_model(linear_model,20)

0.9532 0.9641 0.9659 0.9668 0.9691 0.97 0.9714 0.9723 0.9732 0.9727 0.9736 0.9736 0.9741 0.9736 0.9736 0.9736 0.9736 0.9741 0.975 0.9759 